In [31]:
from init import *

In [32]:
def BuildModel_Simple():
    m = sdk.Model()
    m.TimeStart = 0
    m.TimeSteps = 10
    m.NumPaths = 10000
    m.Add(sdk.IndependentGaussian())
    m.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
    return m

In [33]:
m = BuildModel_Simple()
m.evaluations.append(sdk.EvaluationPoint(0,2))
m.json()

'{"TimeStart": 0, "TimeSteps": 10, "NumPaths": 10000, "updaters": [{"name": "IndependentGaussian", "refs": [], "args": []}, {"name": "BrownianMotion", "refs": [], "args": [0.2, 2], "start": 0.1}], "evaluations": [{"state": 0, "time": 2}]}'

In [34]:
r = requests.post(f'{server}/model',m.json())
r.json()

{'histograms': [],
 'mean': [0.4542025923728943],
 'names': ['BrownianMotion'],
 'npaths': [10000],
 'skewness': [-0.029013577848672867],
 'stddev': [2.836071014404297],
 'time_points': [2.0],
 'time_steps': [10]}

In [35]:
import time
m = BuildModel_Simple()
m.NumPaths = 1000000
m.TimeSteps = 1000
m.RunTimeoutSeconds = 16.7
for t in [1,2,5,10]:
    m.evaluations.append(sdk.EvaluationPoint(0,t))
time_start = time.time()
r = requests.post(f'{server}/model',m.json())
time_end = time.time()
r.json()

{'histograms': [],
 'mean': [0.3015768826007843,
  0.5034372806549072,
  1.1028289794921875,
  2.108328104019165],
 'names': ['BrownianMotion'],
 'npaths': [1000000, 1000000, 1000000, 1000000],
 'skewness': [-0.0017834312748163939,
  -0.00205038720741868,
  -0.002811677986755967,
  -0.003609464503824711],
 'stddev': [2.000115156173706,
  2.829728364944458,
  4.47157096862793,
  6.31917667388916],
 'time_points': [1.0, 2.0, 5.0, 10.0],
 'time_steps': [100, 200, 500, 1000]}

In [36]:
print(f'{m.NumPaths*m.TimeSteps/(time_end-time_start)*1e-9:.1f} nanoseconds to make a single time step')

0.5 nanoseconds to make a single time step


# Two processes, Three evaluations

In [37]:
def TwoProcessesModel ():
    m = sdk.Model()
    m.TimeStart = 0
    m.TimeSteps = 10
    m.NumPaths = 10000
    m.Add(sdk.IndependentGaussian())
    m.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
    m.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
    return m

In [38]:
model = sdk.Model()
model.TimeStart = 0
model.TimeSteps = 10
model.NumPaths = 10000
model.Add(sdk.IndependentGaussian())
model.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
model.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
model.TimeSteps = 10000
for t in [1,2,5]:
    model.evaluations.append(sdk.EvaluationPoint(state=0,time=t))
r = requests.post(f'{server}/model',model.json())
r.json()

{'histograms': [],
 'mean': [0.2743476927280426,
  0.2669976055622101,
  0.4766482412815094,
  0.4413864314556122,
  1.1109087467193604,
  1.0746146440505981],
 'names': ['BrownianMotion', 'BrownianMotion'],
 'npaths': [10000, 10000, 10000, 10000, 10000, 10000],
 'skewness': [0.004301251377910376,
  -0.0032482000533491373,
  0.017873311415314674,
  -0.01917778141796589,
  -0.0011888196459040046,
  -0.0007855305448174477],
 'stddev': [2.0090317726135254,
  1.9885363578796387,
  2.837817668914795,
  2.834514617919922,
  4.4458513259887695,
  4.480775833129883],
 'time_points': [1.0, 2.0, 5.0],
 'time_steps': [2000, 4000, 10000]}

In [39]:
results = run (model)
results.df()

,name,title,state,time,step,npaths,mean,mean_error,stddev,skewness
0,BrownianMotion,,0,1.0,2000,10000,0.294919,0.020170,2.017030,0.015536
1,BrownianMotion,,1,1.0,2000,10000,0.311103,0.020031,2.003073,0.031612
2,BrownianMotion,,0,2.0,4000,10000,0.476229,0.028494,2.849406,0.000286
3,BrownianMotion,,1,2.0,4000,10000,0.508086,0.028257,2.825690,-0.018219
4,BrownianMotion,,0,5.0,10000,10000,1.107325,0.045187,4.518675,0.001883
5,BrownianMotion,,1,5.0,10000,10000,1.155716,0.044591,4.459146,-0.008901
